In [ ]:
import cv2
import numpy as np
import os
import time
from matplotlib import pyplot as plt
from research.utils.image_utils import Picture

import sys
sys.path.append('../q1_o2kr2_dataset_annotations/')
from research_lib.utils.data_access_utils import S3AccessUtils
from research.utils.data_access_utils import RDSAccessUtils

from sqlalchemy import MetaData
import uuid

<h1> Load sample footage </h1>

In [ ]:
f = '/root/data/alok/biomass_estimation/playground/gopro_fish_footage.mp4'
cap = cv2.VideoCapture(f) 

frames = []
count = 0 
while (cap.isOpened()): 
  
    # Capture frame-by-frame 
    ret, frame = cap.read() 
    if frame is not None:
#         frame = cv2.resize(frame, (512, 512)) 
        frames.append(frame)
        
    count += 1
    if count >= 10000:
        break

# release the video capture object 
cap.release() 


<h1> Resize frames and save to disk </h1>

In [ ]:
def adjust_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    
    if value > 0:
        lim = 255 - value
        v[v > lim] = 255
        v[v <= lim] += value
    else:
        adj_value = abs(value)
        lim = adj_value
        v[v <= lim] = 1
        v[v > lim] -= np.array([adj_value], dtype=np.uint8)

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img


output_directory = '/root/data/alok/biomass_estimation/playground/gopro'
for idx, frame in enumerate(frames):
    if idx >= 1800 and idx <= 3300:
        
#     adj_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     dark = adjust_brightness(adj_frame, value=-50)
#     picture = Picture(image_arr=dark)
#     picture.enhance()
#     resized = cv2.resize(picture.get_image_arr(), (512, 512))
#     cv2.imwrite(os.path.join(output_directory, 'frame_{}.jpg'.format(idx)), resized)
        cv2.imwrite(os.path.join(output_directory, 'frame_{}.jpg'.format(idx)), frame)

    if idx % 10 == 0:
        print(idx)

<h1> Upload images to PLALI </h1>

In [ ]:
help(s3.get_matching_s3_keys)

In [ ]:
s3 = S3AccessUtils('/root/data')


In [ ]:
gen = s3.get_matching_s3_keys('aquabyte-images-adhoc', 'alok/playground/gopro/', suffixes=['.jpg'])
keys = [key for key in gen]

In [ ]:
plali_sql_credentials = {
    'host': 'aquabyte-research-db.cfwlu7jbdcqj.eu-west-1.rds.amazonaws.com',
    'port': 5432,
    'user': 'plali',
    'password': 'amethyst that lift dune',
    'database': 'aquabyte_prod'
}

In [ ]:
 

def establish_plali_connection():
    rds = RDSAccessUtils(plali_sql_credentials)
    engine = rds.sql_engine
    sql_metadata = MetaData()
    sql_metadata.reflect(bind=engine)
    return engine, sql_metadata


def process_into_plali_records(s3_urls):

    values_to_insert = []
    for idx, s3_url in enumerate(s3_urls):
        id = str(uuid.uuid4())
        images = {s3_url}
        metadata = {'name': 'gopro_annotations'}
        priority = float(len(s3_urls) - idx) / len(s3_urls)

        values = {
            'id': id,
            'workflow_id': '00000000-0000-0000-0000-000000000057',
            'images': images,
            'metadata': metadata,
            'priority': priority
        }

        values_to_insert.append(values)

    return values_to_insert


def insert_into_plali(values_to_insert, engine, sql_metadata):
    table = sql_metadata.tables['plali_images']
    conn = engine.connect()
    trans = conn.begin()
    conn.execute(table.insert(), values_to_insert)
    trans.commit()

In [ ]:
engine, sql_metadata = establish_plali_connection()

In [ ]:
s3_urls = [os.path.join('s3://', 'aquabyte-images-adhoc', key) for key in keys]
values_to_insert = process_into_plali_records(s3_urls)

In [ ]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

n = 10
count = 0
for chunk in chunker(values_to_insert[2:], n):
    insert_into_plali(chunk, engine, sql_metadata)
    
    count += 1
    print(count)

<h1> Load production YOLO model </h1>

In [ ]:
import sys

In [ ]:
sys.path.append('/root/alok/repos/production_algo/segmentation/src')

In [ ]:
from pathlib import Path
from yolo3 import predict as yolo3


def generate_prediction():
    predictor = yolo3.Predictor("yolo3-20200114", "/root/data/alok/biomass_estimation/playground")

    items = []
    for count in range(2000, 5000):
#     for count in range(1):
        item = yolo3.Item(
            id=str(count),
#             image_path=Path('/root/data/alok/biomass_estimation/playground/left_frame.resize_512_512.jpg')
            image_path=Path(os.path.join(output_directory, 'frame_{}.jpg'.format(count)))
        )
        items.append(item)
    
    print(len(items))
    
    yolo3.load_images(items)
    result = predictor.predict(items)
    return result




In [ ]:
result = generate_prediction()

In [ ]:
total = 0
for res in result:
    total += len(res['bboxes'])

print(total)

In [ ]:
plt.imshow(cv2.imread('/root/data/alok/biomass_estimation/playground/left_frame.resize_512_512.jpg'))

In [ ]:
im = cv2.imread('/root/data/alok/biomass_estimation/playground/gopro/frame_2100.jpg')
# bright_im = adjust_brightness(im, value=-50)

In [ ]:
plt.imshow(im)